# LSTM의 `return_sequences`, `return_state` 이해하기
* `return_sequences` : 매 타임 스텝의 출력 여부 결정 결정
* `return_state` : 제일 마지막 스테이트 출력 여부 결정

In [ ]:
import numpy as np
from tensorflow.keras.layers import LSTM

In [ ]:
sample_train = np.random.randn(1, 4, 5) # (N, L, I) : (데이터개수, 최대타임스탭, 임베딩차원)

`return_sequences=False`, `return_state=False`

In [ ]:
# 제일 마지막 hidden state만 반환된다.
last_hidden_state = LSTM(3, return_sequences=False, return_state=False)(sample_train)
print(last_hidden_state)

tf.Tensor([[0.1270768  0.5852918  0.05390989]], shape=(1, 3), dtype=float32)


`return_sequences=False`, `return_state=True`

* `hidden_states` = `last_hidden_state`
* `last_cell_state`

In [ ]:
hidden_states, last_hidden_state, last_cell_state = LSTM(3, return_sequences=False, return_state=True)(sample_train)
print("hidden_states : {}".format(hidden_states))
print("last_hidden_state : {}".format(last_hidden_state))
print("last_cell_state : {}".format(last_cell_state))

hidden_states : [[-0.1230509  -0.12402871 -0.01906718]]
last_hidden_state : [[-0.1230509  -0.12402871 -0.01906718]]
last_cell_state : [[-0.18105246 -0.13825296 -0.03515929]]


`return_sequences=True`, `return_state=False`

In [ ]:
hidden_states = LSTM(3, return_sequences=True, return_state=False)(sample_train)
print("hidden_states : {} / shape : {}".format(hidden_states, hidden_states.shape))

hidden_states : [[[-0.11384317 -0.04172194  0.04565322]
  [-0.28455564 -0.43353674 -0.00277211]
  [-0.2895847  -0.4830808  -0.32144073]
  [-0.18519112 -0.3412207   0.03781172]]] / shape : (1, 4, 3)


`return_sequences=True`,`return_state=True`

In [ ]:
hidden_states, last_hidden_state, last_cell_state = LSTM(3, return_sequences=True, return_state=True)(sample_train)
print("hidden_states : {}".format(hidden_states))
print("last_hidden_state : {}".format(last_hidden_state))
print("last_cell_state : {}".format(last_cell_state))

hidden_states : [[[ 0.17464027  0.20474215 -0.16715199]
  [-0.04145287 -0.03030119 -0.26777637]
  [-0.09709232 -0.16504493 -0.19007929]
  [-0.04109285 -0.10302802 -0.32218415]]]
last_hidden_state : [[-0.04109285 -0.10302802 -0.32218415]]
last_cell_state : [[-0.45237607 -0.7649007  -0.5107193 ]]


# Seq2Seq로 챗봇 만들기

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 218 kB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 448 kB 66.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import random # 나중에 데이터 셔플링 할 예정
import tensorflow as tf
from konlpy.tag import Okt

## 하이퍼 파라미터

In [ ]:
num_epochs=200
vocab_size=2000

# Encoder

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self):
    super(Encoder, self).__init__()
    self.emb  = tf.keras.layers.Embedding(vocab_size, 64)

    # 제일 마지막 state를 리턴해야 context vector가 나옴!
    self.lstm = tf.keras.layers.LSTM(512, return_sequences=False, return_state=True)

  def call(self, x, training=False):
    # 임베딩
    x = self.emb(x)

    # Encoder에서는 context vector만 얻어내면 되기 때문에 각 time 별 state는 필요가 없다.
    _, h, c = self.lstm(x)

    # context vector return
    return h, c

# Decoder

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self):
    super(Decoder, self).__init__()
    self.emb = tf.keras.layers.Embedding(vocab_size, 64)
    self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)

    # 각 셀마다 2000개의 output을 내고, 어떤 단어가 추정 되었을지를 계산
    self.dense = tf.keras.layers.Dense(vocab_size, activation='softmax')

  def call(self, inputs, training=False):
    x, h, c = inputs # shifted, hidden_state, cell_state
    x = self.emb(x) # 입력한 단어에 대한 임베딩 벡터

    # y_ : 해당 시퀀스의 hidden_state
    y_, h, c = self.lstm(x, initial_state=[h, c]) # initial_state : 초기화 할 hidden_state, cell_state를 지정

    y = self.dense(y_)

    return y, h, c

# Seq2Seq

In [ ]:
class Seq2seq(tf.keras.Model):
  
  def __init__(self, sos, eos):
    super(Seq2seq, self).__init__()
    self.sos = sos # decoder에서 사용되어질 sos
    self.eos = eos # encoder에서 사용되어질 eos

    self.enc = Encoder()
    self.dec = Decoder()

  def call(self, inputs, training=False):
    
    if training: # 훈련에서는 Teacher Forcing 때문에 정답이 들어옴

      x, y = inputs # (output_labels, shifted labels)
      h, c = self.enc(x) # context vector가 등장
      
      y, _, __ = self.dec((y, h, c)) # teacher forcing. Decoder의 입력으로 Shifted Output을 넣어줌
      
      return y
    else: # 테스트 할 때는 x만 들어 온다..
      x = inputs
      h, c = self.enc(x) # last_cell_state, last_hidden_state
      
      # <sos> 입력
      # <sos> 토큰을 tensor 배열화 시켜야 함
      y = tf.convert_to_tensor(self.sos) # 0 rank tensor로 변환
      y = tf.reshape(y, (1, 1)) # <sos>가 (1, 1)형식으로 변환. ( 1배치, 1타임 스텝)을 의미. embedding 레이어에 넣을 예정

      # 최대 입력 길이 만큼의 공간을 미리 만들어 놓자
      seq = tf.TensorArray(tf.int32, 64) # 64개의 텐서 배열 만들어 놓기

      # tensorflow의 session 환경에서 for문을 조금 더 빠르게 돌릴 수 있다!
      for idx in tf.range(64):
        # 제일 처음엔 <sos>, 인코더의 h, c (context vector)가 들어간다.
        ############################################################
        y, h, c = self.dec([y, h, c]) # 리턴 받는 y는 softmax의 결과
        ############################################################
        y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)

                                  # 한 문장 들어 가서 하나의 결과. 1배치가 들어가니까 나오는 것도 1개
        y = tf.reshape(y, (1, 1)) # (1 배치, 1단어를 의미하기 위해 reshape - 테스트 할 때는 배치를 1로 설정할 예정..)

        seq = seq.write(idx, y) # 순서대로 write

        if y == self.eos:
          break
      
      return tf.reshape(seq.stack(), (1, 64))

# 학습, 테스트 루프 정의

In [ ]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuarcy):
  # labels는 <sos>, <eos> 를 포함한 정보
  # output_labels : <sos>를 제외하고 <eos>를 포함해서 만든다.
  output_labels = labels[:, 1:]
  # shifted_lables : <sos>를 포함하고 <eos>를 제외해서 만든다.
  shifted_labels = labels[:, :-1]

  with tf.GradientTape() as tape:
    # inputs : x의 역할. Encoder에 들어감
    # shifted_labels : Encoder가 예측하고, 예측해야 할 데이터
    predictions = model([inputs, shifted_labels], training=True) # 예측을 하고
    loss = loss_object(output_labels, predictions) # 정답이 이거였어~ 라고 이야기 하는 것
  
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  
  train_loss(loss)
  train_accuracy(output_labels, predictions)

@tf.function
def test_step(model, inputs):
  # 입력 데이터만 주고 추론은 모델이 알아서 할 수 있도록...
  return model(inputs, training=False)

# 데이터셋 준비
* http://www.aihub.or.kr

In [ ]:
from konlpy.tag import Okt

dataset_file = "chatbot_data.csv"
okt = Okt()

In [ ]:
with open(dataset_file, 'r') as file:
  lines = file.readlines()
  seq = [" ".join(okt.morphs(line)) for line in lines]

In [ ]:
seq[:6]

['아이스 아메리카노 하나요 \n',
 '테이크아웃 하실 건가 요 ? \n',
 '저 카푸치노 로 주문 할게요 \n',
 '시럽 은 얼마나 뿌려 드릴 까요 ? \n',
 '저 도장 다 모았는데 나중 에 써도 되나요 ? \n',
 '네 다음 에 써도 됩니다 \n']

In [ ]:
questions = seq[::2]
answers = ["\t " + lines for lines in seq[1::2]] # \t : <sos>

print(questions[:3])
print(answers[:3])

['아이스 아메리카노 하나요 \n', '저 카푸치노 로 주문 할게요 \n', '저 도장 다 모았는데 나중 에 써도 되나요 ? \n']
['\t 테이크아웃 하실 건가 요 ? \n', '\t 시럽 은 얼마나 뿌려 드릴 까요 ? \n', '\t 네 다음 에 써도 됩니다 \n']


# 데이터 잘라내기

In [ ]:
num_samples = len(questions)
print(num_samples)

500


In [ ]:
term = list(range(num_samples))
print("섞이기 전 : {}".format(term[:10]))
# 랜덤 시드 고정
random.seed(0)
random.shuffle(term)

print("섞인 후 : {}".format(term[:10]))

섞이기 전 : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
섞인 후 : [419, 459, 130, 431, 370, 26, 201, 56, 366, 108]


* questions : 입력 데이터(inputs)
* answers : 예측 레이블 (outputs)

In [ ]:
train_q = [] # X_train
train_a = [] # y_train

test_q = [] # X_test
test_a = [] # y_test

In [ ]:
test_ratio = 0.2
test_cnt = int(len(questions) * test_ratio)

train_indices = term[test_cnt: ]
test_indices  = term[:test_cnt]

for idx in train_indices:
  train_q.append(questions[idx])
  train_a.append(answers[idx])

for idx in test_indices:
  test_q.append(questions[idx])
  test_a.append(answers[idx])

In [ ]:
test_q[:3], test_a[:3]

(['사이 즈 업 해서 주세요 \n',
  '캐러멜 드리블 이랑 통 잡아 칩이요 \n',
  '시즌 메뉴 와 함께 구성 되어 있는 세트 메뉴 가 있나요 ? \n'],
 ['\t 네 결제 는 어떻게 도 와 드릴 까요 ? \n',
  '\t 6700원 결제 도 와 드리겠습니다 \n',
  '\t 네 치즈 케이크 와 시즌 메뉴 두 잔 으로 구성 된 세트 메뉴 있습니다 \n'])

# 토크나이징

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

In [ ]:
tokenizer.fit_on_texts(train_q + train_a) # 질문과 대답의 모든 내용을 토큰화
print(tokenizer.word_index)

{'\n': 1, '\t': 2, '네': 3, '주세요': 4, '로': 5, '아메리카노': 6, '는': 7, '에': 8, '아이스': 9, '도': 10, '요': 11, '잔': 12, '이': 13, '한': 14, '드릴': 15, '까요': 16, '은': 17, '입니다': 18, '사이즈': 19, '가': 20, '있나요': 21, '결제': 22, '수': 23, '하나': 24, '있습니다': 25, '와': 26, '드시고': 27, '해주세요': 28, '할게요': 29, '으로': 30, '라테': 31, '추가': 32, '따뜻한': 33, '주문': 34, '사용': 35, '음료': 36, '되나요': 37, '여기': 38, '아니요': 39, '거': 40, '얼마': 41, '개': 42, '그럼': 43, '카드': 44, '랑': 45, '드리겠습니다': 46, '케이크': 47, '어떤': 48, '걸': 49, '포인트': 50, '가시나요': 51, '한잔': 52, '할인': 53, '적립': 54, '다': 55, '커피': 56, '더': 57, '인가요': 58, '쿠폰': 59, '가요': 60, '드릴게요': 61, '티': 62, '건': 63, '가능합니다': 64, '알겠습니다': 65, '에서': 66, '가능한가요': 67, '매장': 68, '를': 69, '진동': 70, '면': 71, '벨': 72, '안': 73, '번호': 74, '만': 75, '에요': 76, '메뉴': 77, '하나요': 78, '디카': 79, '페인': 80, '건가': 81, '샷': 82, '있어요': 83, '됩니다': 84, '테이크아웃': 85, '예요': 86, '스무디': 87, '게': 88, '카페라테': 89, '두': 90, '같이': 91, '자몽': 92, '하고': 93, '치즈케이크': 94, '제일': 95, '뭐': 96, '카페모카': 97, '기프티콘': 98, '세트':

정수 인코딩

In [ ]:
train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq  = tokenizer.texts_to_sequences(test_q)
test_a_seq  = tokenizer.texts_to_sequences(test_a)

train_q_seq[:3], train_a_seq[:3]

([[85, 12, 30, 4, 1], [3, 239, 1], [3, 300, 301, 47, 4, 1]],
 [[2, 627, 628, 629, 73, 66, 630, 631, 35, 113, 23, 378, 1],
  [2, 57, 162, 63, 139, 60, 1],
  [2, 36, 7, 227, 5, 15, 16, 1]])

패딩 후 최종 데이터 마련하기

In [ ]:
# 문장의 최대길이 64로 설정 했음!
X_train = pad_sequences(
    train_q_seq,
    value=0,
    padding='pre',
    maxlen=64
)

y_train = pad_sequences(
    train_a_seq,
    value=0,
    padding='post',
    maxlen=65 # <sos>, <eos>
)

X_test = pad_sequences( test_q_seq, value=0, padding='pre', maxlen=64 )
y_test = pad_sequences( test_a_seq, value=0, padding='post', maxlen=65 )

In [ ]:
X_train[0], y_train[0]

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 85, 12, 30,  4,  1], dtype=int32),
 array([  2, 627, 628, 629,  73,  66, 630, 631,  35, 113,  23, 378,   1,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       dtype=int32))

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(32).prefetch(1024) # prefetch : 데이터를 미리 저장할 공간을 의미
test_ds  = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(1).prefetch(1024)

# 학습 환경 정의
모델 생성, 손실 함수, 최적화 알고리즘, 평가지표 정의

In [ ]:
# 모델 생성
model = Seq2seq(
    sos=tokenizer.word_index["\t"],
    eos=tokenizer.word_index["\n"]
)

# Loss 선정. 정수 인코딩된 결과를 t로 사용, softmax 이용한 정수값을 예측으로 쓰니까 sparse_categorical_crossentropy
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# 모델 평가 방식
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# 학습 루프 동작

In [ ]:
EPOCHS = 200
for epoch in range(EPOCHS):
  for seqs, labels in train_ds:
    train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)
  
  print("Epoch : {}, Loss : {:.3f}, Accuracy : {:.3f}".format(epoch + 1,
                                                      train_loss.result(),
                                                      train_accuracy.result() * 100))
  
  train_loss.reset_states()
  train_accuracy.reset_states()

Tensor("strided_slice_1:0", shape=(32, 64), dtype=int32)
Tensor("strided_slice_1:0", shape=(32, 64), dtype=int32)
Tensor("strided_slice_1:0", shape=(16, 64), dtype=int32)
Epoch : 1, Loss : 3.192, Accuracy : 83.109
Epoch : 2, Loss : 0.605, Accuracy : 90.836
Epoch : 3, Loss : 0.575, Accuracy : 91.109
Epoch : 4, Loss : 0.562, Accuracy : 91.094
Epoch : 5, Loss : 0.552, Accuracy : 91.164
Epoch : 6, Loss : 0.540, Accuracy : 91.090
Epoch : 7, Loss : 0.544, Accuracy : 91.090
Epoch : 8, Loss : 0.534, Accuracy : 91.137
Epoch : 9, Loss : 0.530, Accuracy : 91.176
Epoch : 10, Loss : 0.521, Accuracy : 91.184
Epoch : 11, Loss : 0.515, Accuracy : 91.328
Epoch : 12, Loss : 0.501, Accuracy : 91.293
Epoch : 13, Loss : 0.487, Accuracy : 91.477
Epoch : 14, Loss : 0.471, Accuracy : 91.754
Epoch : 15, Loss : 0.457, Accuracy : 91.992
Epoch : 16, Loss : 0.445, Accuracy : 92.238
Epoch : 17, Loss : 0.431, Accuracy : 92.391
Epoch : 18, Loss : 0.429, Accuracy : 92.492
Epoch : 19, Loss : 0.425, Accuracy : 92.496
Ep

Accuracy가 좋은 이유는?? `Teacher Forcing` 했으니까 좋을 수 밖에..

# 테스트 루프 만들기

In [ ]:
for test_seq, test_labels in test_ds:
  prediction = test_step(model, test_seq)
  
  test_q = tokenizer.sequences_to_texts(test_seq.numpy()) # 질문
  test_a = tokenizer.sequences_to_texts(test_labels.numpy()) # 실제 대답
  test_p = tokenizer.sequences_to_texts(prediction.numpy()) # 챗봇의 대답

  print("______")
  print("질문 : \t{}".format(test_q))
  print("실제 대답 : {}".format(test_a))
  print("챗봇 대답 : {}".format(test_p))


______
질문 : 	['사이 즈 업 해서 주세요 \n']
실제 대답 : ['\t 네 결제 는 어떻게 도 와 드릴 까요 \n']
챗봇 대답 : ['다른 건 필요 없으신 가요 \n']
______
질문 : 	['캐러멜 드리블 이랑 통 잡아 \n']
실제 대답 : ['\t 결제 도 와 드리겠습니다 \n']
챗봇 대답 : ['네 가능합니다 \n']
______
질문 : 	['시즌 메뉴 와 함께 되어 있는 세트 메뉴 가 있나요 \n']
실제 대답 : ['\t 네 치즈 케이크 와 시즌 메뉴 두 잔 으로 세트 메뉴 있습니다 \n']
챗봇 대답 : ['네 시즌 메뉴 가능합니다 \n']
______
질문 : 	['아메리카노 1 잔 주세요 \n']
실제 대답 : ['\t 매장 에서 드시고 가시나요 \n']
챗봇 대답 : ['따뜻한 걸 로 드릴 까요 \n']
______
질문 : 	['그럼 와 아이스 아메리카노 로 할게요 \n']
실제 대답 : ['\t 더 필요하신 건 없나요 \n']
챗봇 대답 : ['네 결제 는 어떤 것 으로 도 와 드릴 까요 \n']
______
질문 : 	['밀크 티 있나요 \n']
실제 대답 : ['\t 네 있습니다 \n']
챗봇 대답 : ['아니요 한 사이즈 로만 판매 하고 있습니다 \n']
______
질문 : 	['네 기프티콘 여기 있어요 \n']
실제 대답 : ['\t 아메리카노 기프티콘 사용 되었습니다 \n']
챗봇 대답 : ['네 텀블러 할인 되셔서 4000원 결제 도 와 드리겠습니다 \n']
______
질문 : 	['네 오늘 의 커피 로 주세요 \n']
실제 대답 : ['\t 네 사이즈 는 어떤 걸 로 주문 넣어 드릴 까요 \n']
챗봇 대답 : ['네 음료 와 카페모카 입니다 \n']
______
질문 : 	['포인트 사용 없이 적립 만 할게요 \n']
실제 대답 : ['\t 네 멤버십 카드 주시 면 도 와 드리겠습니다 \n']
챗봇 대답 : ['매장 에서 드시고 가세 요 \n']
______
질문 : 	['네 감사합니다 \n']
실제